<h1 align='center'>
<br>
<img src="assets/IBM-Logo.png" alt="IBM Course" width="200">
<br>
    Applied Data Science Capstone
</h1>

<h2 align='center'><font size=5>
    Week 3 Peer-graded Assignment - Segmenting and Clustering Neighborhoods in Toronto | Part II</font>
</h2>


## Scraping Data from Wikipedia

Before starting, let's first import all the libraries needed for the project.

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt # plotting library
%matplotlib inline 

# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library

# showing progress
from tqdm.notebook import tqdm

plt.style.use('ggplot') # refined style

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
tables = pd.read_html(url)
can_df = tables[0]

In [3]:
can_df = can_df[~(can_df['Borough'] == 'Not assigned')]
can_df.reset_index(inplace=True)
can_df = can_df.drop('index', 1)

In [4]:
can_df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Let's make sure that postal code appears only once in the entire data set.

In [5]:
len(set(can_df['Postal Code'])) == len(can_df['Postal Code'])

True

Also, let's check if there is still `Not assigned` neighborhood.

In [6]:
can_df[can_df['Neighbourhood']== 'Not assigned'].shape[0]

0

In [7]:
can_df.shape

(103, 3)

## Adding Latitude and Longtitude

In [8]:
geo_data = pd.read_csv('assets/Geospatial_Coordinates.csv')
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
tor_df = can_df.merge(geo_data, on='Postal Code')
tor_df.head(12)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
